<a href="https://colab.research.google.com/github/emailic/YOLOv5-Microplasticos/blob/main/VGG16_classification_SUCCESS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://medium.com/nerd-for-tech/image-classification-using-transfer-learning-vgg-16-2dc2221be34c

In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
print(tf. __version__) 

2.8.2


In [ ]:
base_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset'
train_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/train'
train_benign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/train/benign'
train_malign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/train/malignant'
test_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/test'
test_benign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/test/benign'
test_malign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/test/malignant'
valid_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation'
valid_benign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation/benign'
valid_malign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation/malignant'


In [ ]:
tr_0='/content/drive/My Drive/microplasticos/images/train_vgg16/0'
tr_1='/content/drive/My Drive/microplasticos/images/train_vgg16/1'
tr_2='/content/drive/My Drive/microplasticos/images/train_vgg16/2'
tr_3='/content/drive/My Drive/microplasticos/images/train_vgg16/3'
tr_4='/content/drive/My Drive/microplasticos/images/train_vgg16/4'
val_0='/content/drive/My Drive/microplasticos/images/val_vgg16/0'
val_1='/content/drive/My Drive/microplasticos/images/val_vgg16/1'
val_2='/content/drive/My Drive/microplasticos/images/val_vgg16/2'
val_3='/content/drive/My Drive/microplasticos/images/val_vgg16/3'
val_4='/content/drive/My Drive/microplasticos/images/val_vgg16/4'
te_0='/content/drive/My Drive/microplasticos/images/test_vgg16/0'
te_1='/content/drive/My Drive/microplasticos/images/test_vgg16/1'
te_2='/content/drive/My Drive/microplasticos/images/test_vgg16/2'
te_3='/content/drive/My Drive/microplasticos/images/test_vgg16/3'
te_4='/content/drive/My Drive/microplasticos/images/test_vgg16/4'
train_dir='/content/drive/My Drive/microplasticos/images/train_vgg16'
test_dir='/content/drive/My Drive/microplasticos/images/test_vgg16'
valid_dir='/content/drive/My Drive/microplasticos/images/val_vgg16'
base_dir='/content/drive/My Drive/microplasticos/images'

Data Cleaning: Eliminating images which cannot be opened.

In [ ]:
dir0=os.listdir(te_0)
dir1=os.listdir(te_1)
dir2=os.listdir(te_2)
dir3=os.listdir(te_3)
dir4=os.listdir(te_4)
counter=0
for (dir, tr) in [(dir0, te_0),(dir1,te_1),(dir2,te_2),(dir3,te_3),(dir4,te_4)]:
  for image in dir :
    try:
      im=Image.open(tr+'/'+image)
      print('verified ', tr+'/'+image )
    except:
      counter+=1
      print('Deleted', tr+'/'+image, counter) #train_dir+'/'+x+'/'+image
      os.remove(tr+'/'+image)

    #delete exceptions



verified  /content/drive/My Drive/microplasticos/images/test_vgg16/0/20220516_113208.jpg_grid_0_0.jpg_tblr_607.65_838.42_421.22_591.41.jpg
verified  /content/drive/My Drive/microplasticos/images/test_vgg16/0/20220516_113208.jpg_grid_384_384.jpg_tblr_431.37_623.68_350.71_486.29.jpg
verified  /content/drive/My Drive/microplasticos/images/test_vgg16/0/20220516_113208.jpg_grid_384_384.jpg_tblr_123.68_312.14_316.11_428.61.jpg
verified  /content/drive/My Drive/microplasticos/images/test_vgg16/0/20220516_113208.jpg_grid_384_0.jpg_tblr_777.56_935.25_708.72_873.15.jpg
verified  /content/drive/My Drive/microplasticos/images/test_vgg16/0/20220516_113208.jpg_grid_384_0.jpg_tblr_927.53_1115.99_524.13_665.47.jpg
verified  /content/drive/My Drive/microplasticos/images/test_vgg16/0/20220516_113208.jpg_grid_384_0.jpg_tblr_546.72_785.18_373.57_604.34.jpg
verified  /content/drive/My Drive/microplasticos/images/test_vgg16/0/20220516_113208.jpg_grid_384_0.jpg_tblr_123.68_312.14_700.11_812.61.jpg
verified  

In [ ]:
'''num_benign_train = len(os.listdir(train_benign_dir))
num_malignant_train = len(os.listdir(train_malign_dir))
num_benign_validaition = len(os.listdir(valid_benign_dir))
num_malignant_validation= len(os.listdir(valid_malign_dir))
num_benign_test = len(os.listdir(test_benign_dir))
num_malignant_test= len(os.listdir(test_malign_dir))'''


In [ ]:

'''

print("Total Training Benign Images",num_benign_train)
print("Total Training Malignant Images",num_malignant_train)
print("--")
print("Total validation Benign Images",num_benign_validaition)
print("Total validation Malignant Images",num_malignant_validation)
print("--")print("Total Test Benign Images", num_benign_test)
print("Total Test Malignant Images",num_malignant_test)
total_train = num_benign_train+num_malignant_train
total_validation = num_benign_validaition+num_malignant_validation
total_test = num_benign_test+num_malignant_test
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)
'''

In [ ]:

IMG_SHAPE  = 224
batch_size = 32


image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'categorical')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=valid_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='categorical')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='categorical')


Found 6610 images belonging to 5 classes.
Found 1531 images belonging to 5 classes.
Found 642 images belonging to 5 classes.


In [ ]:

pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x) #was advided to put 3 instead of 5 


#x = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(pre_trained_model.input, x)


model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])


input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11445833812042921560
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14444920832
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16196299554546014164
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(6610//batch_size),
epochs = 5,
validation_data=val_data_gen,
validation_steps=(1531//batch_size),
batch_size = batch_size,
workers=16,
use_multiprocessing=True,
verbose = 1)


Epoch 1/5
206/206 [==============================] - ETA: 0s - loss: 1.0996 - acc: 0.5745 

In [ ]:


result = model.evaluate(test_data_gen,batch_size=batch_size)
print("test_loss, test accuracy",result)


21/21 [==============================] - 4s 201ms/step - loss: 0.8987 - acc: 0.6822
test_loss, test accuracy [0.8986883163452148, 0.6822429895401001]


In [ ]:

model_json = model.to_json()
with open("/content/drive/MyDrive/microplasticos/models/vgg16_trained.json", "w") as json_file:
  json_file.write(model_json)
model.save("/content/drive/MyDrive/microplasticos/models/VGG16_Classifier.h5")
print("Saved model to disk")
model.save_weights("/content/drive/MyDrive//microplasticos/models/VGG16_weights.h5")

Saved model to disk
